In [ ]:
import logging
from multimodal_VAAL_solver import multi_modal_VAAL_Solver
import torch
from torchvision import datasets, transforms
import torch.utils.data.sampler  as sampler
import torch.utils.data as data
import torch.nn as nn
import numpy as np
import random
from pathlib import Path
from custom_datasets import *
import model
import multi_modal_model
from VAAL_solver import VAAL_Solver
from utils import *
import arguments
from evaluate import evaluate, evaluate_classwise
from unet import UNet
import wandb
import torch.backends.cudnn as cudnn
import warnings
warnings.filterwarnings("ignore")


# pick an image to test 

# some image transformations 
transform_x = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.5,0.5,0.5],
            std=[0.5,0.5,0.5])
        ])
transform_y = transforms.Compose([
    transforms.ToTensor()
])

transform_z = transforms.Compose([          
    transforms.ToTensor()
])


def decode_target(self, target):
    h,w = target.shape
    
    new_mask = torch.zeros(h, w, 3, dtype=torch.uint8)
    for k in self.train_ids_to_color:
        # Get all indices for current class
        idx = (target==torch.tensor(k, dtype=torch.long))
        new_mask[idx] = torch.tensor(self.train_ids_to_color[k], dtype=torch.uint8)
    return new_mask

def preprocess(image, mask, depth= None, transform = None, target_transform = None, depth_transform = None, flip=False, resize = None, scale=None, crop=None):
    if flip:
        if random.random() < 0.5:
            image = image.transpose(Image.FLIP_LEFT_RIGHT)
            mask = mask.transpose(Image.FLIP_LEFT_RIGHT)
            if depth is not None:
                depth = depth.transpose(Image.FLIP_LEFT_RIGHT)

    
    if scale :
        w, h = image.size
        rand_log_scale = math.log(scale[0], 2) + random.random() * (math.log(scale[1], 2) - math.log(scale[0], 2))
        random_scale = math.pow(2, rand_log_scale)
        new_size = (int(round(w * random_scale)), int(round(h * random_scale)))
        image = image.resize(new_size, Image.ANTIALIAS)
        mask = mask.resize(new_size, Image.NEAREST)
        if depth is not None:
            depth = depth.resize(new_size, Image.NEAREST)

    if resize:
        image = image.resize((resize,resize), Image.ANTIALIAS)
        mask = mask.resize((resize,resize), Image.NEAREST)
        
        if depth is not None:
            depth = depth.resize((resize,resize), Image.NEAREST)


    if transform:
        image = transform(image)
    if target_transform:
        mask = 255 * target_transform(mask)

    if depth is not None:
        if depth_transform:
            depth = target_transform(depth)

    mask = mask.long()
    if depth is not None:
        return image, mask, depth
    else:
        return image, mask, 0



def test_task(scale, net, train_loader, val_loader, test_loader,wandb_log = None, split = 1):


    scale = tuple(float(i) for i in args.scale.split(","))
    if min(scale) == 0:
        scale = None

    dir_checkpoint = Path('./checkpoints/')
    
    
    net.load_state_dict(torch.load(str(dir_checkpoint)+'/'+args.expt + '/'+ 'checkpoint'+str(split)+'.pth'))
    
    # val_score,_,_,_ = evaluate_classwise(net, val_loader, args.device)
    # for key, value in val_score.items():
    #         wandb_log.log({'Val Dice Class '+str(key): value})

    mean_dice_score,test_score, SME, class_wise_SME, classwise_perbatch_dice_scores = evaluate_classwise(net, test_loader, args.device, log_masks = wandb_log)
    classwise_perbatch_dice_scores_list = [v for k,v in classwise_perbatch_dice_scores.items()]


    for key, value in test_score.items():
            wandb_log.log({'Test Dice Class '+str(key): value})
            #wandb_log.log({'Test SME Class '+str(key): class_wise_SME[key]})
            wandb_log.log({'Test STD Class '+str(key): class_wise_SME[key]})
            wandb_log.log({'Test per class dice Class '+str(key): classwise_perbatch_dice_scores_list[key]})

    #wandb_log.log({'Test SME ':SME})
    wandb_log.log({'Test STD ':SME})
    wandb_log.log({'Test Mean Dice Score'+str(key): value})

if __name__ == '__main__':
    args = arguments.get_args()
    fix_seed(0)
    main(args)

